<a href="https://colab.research.google.com/github/nurniahamid/movie_recommender/blob/main/movie_recomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pertanyaan Bisnis**

1. Bagaimana memberikan rekomendasi film yang relevan kepada pengguna berdasarkan preferensi genre yang dimilikinya?

2. Bagaimana menyarankan film kepada pengguna dengan mempertimbangkan perilaku pengguna lain yang memiliki preferensi serupa?


Goals

1. Membangun sistem rekomendasi berbasis konten (content-based filtering) yang dapat menganalisis informasi film (seperti genre) untuk memberikan saran yang sesuai dengan riwayat tontonan pengguna.

2. Mengembangkan sistem rekomendasi berbasis kolaboratif (collaborative filtering) yang dapat memberikan saran kepada pengguna berdasarkan pola rating dan preferensi dari pengguna lain yang serupa.

In [4]:
import pandas as pd

# **Data Understanding**

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nurniahamid","key":"f5ac50c26628fb88a41f246b68d97d62"}'}

In [6]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d gargmanas/movierecommenderdataset
!unzip movierecommenderdataset.zip -d movierecommenderdataset

Dataset URL: https://www.kaggle.com/datasets/gargmanas/movierecommenderdataset
License(s): GPL-2.0
  0% 0.00/846k [00:00<?, ?B/s]
100% 846k/846k [00:00<00:00, 251MB/s]
Archive:  movierecommenderdataset.zip
  inflating: movierecommenderdataset/movies.csv  
  inflating: movierecommenderdataset/ratings.csv  


In [7]:
movies = pd.read_csv('movierecommenderdataset/movies.csv')
ratings = pd.read_csv('movierecommenderdataset/ratings.csv')

print('Jumlah data film: ', len(movies.movieId.unique()))
print('Jumlah data penilaian yang diberikan pengguna: ', len(ratings.movieId.unique()))

Jumlah data film:  9742
Jumlah data penilaian yang diberikan pengguna:  9724


Variabel dalam data :
- Movies : merupakan jumlah data film
- rating : merupakan data rating pengguna terhadap film

# **Univariate Exploratory Data Analysis**

In [8]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [9]:
#variabel movies
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


melihat ada berapa banyak data berdasarkan movieId dan untuk mengetahui jenis-jenis genre yang ada dalam data

In [10]:
print('Banyak Data : ', len(movies.movieId.unique()))
print('Jumlah Jenis Genre film : ', len(movies.genres.unique()))
print('Jenis Genre film: ', movies.genres.unique())

Banyak Data :  9742
Jumlah Jenis Genre film :  951
Jenis Genre film:  ['Adventure|Animation|Children|Comedy|Fantasy'
 'Adventure|Children|Fantasy' 'Comedy|Romance' 'Comedy|Drama|Romance'
 'Comedy' 'Action|Crime|Thriller' 'Adventure|Children' 'Action'
 'Action|Adventure|Thriller' 'Comedy|Horror'
 'Adventure|Animation|Children' 'Drama' 'Action|Adventure|Romance'
 'Crime|Drama' 'Drama|Romance' 'Action|Comedy|Crime|Drama|Thriller'
 'Comedy|Crime|Thriller' 'Crime|Drama|Horror|Mystery|Thriller'
 'Drama|Sci-Fi' 'Children|Drama' 'Adventure|Drama|Fantasy|Mystery|Sci-Fi'
 'Mystery|Sci-Fi|Thriller' 'Children|Comedy' 'Drama|War'
 'Action|Crime|Drama' 'Action|Adventure|Fantasy' 'Comedy|Drama|Thriller'
 'Mystery|Thriller' 'Animation|Children|Drama|Musical|Romance'
 'Crime|Mystery|Thriller' 'Adventure|Drama' 'Drama|Thriller'
 'Comedy|Crime' 'Action|Sci-Fi|Thriller' 'Action|Comedy|Horror|Thriller'
 'Comedy|Drama' 'Documentary' 'Action|Crime|Drama|Thriller'
 'Crime|Drama|Romance' 'Action|Adventure|Dram

In [11]:
# variabel ratings
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [12]:
print('Banyaknya data rating : ', len(ratings.userId.unique()))
print('Banyaknya film yang di rating : ', len(ratings.movieId.unique()))

Banyaknya data rating :  610
Banyaknya film yang di rating :  9724


In [13]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


# **Data Preprocessing**

#### Menggabungkan data movies dan ratings

In [14]:
merged_df = pd.merge(ratings, movies, on='movieId', how='left')
merged_df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),Drama|Horror|Thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),Action|Crime|Thriller
100833,610,168250,5.0,1494273047,Get Out (2017),Horror
100834,610,168252,5.0,1493846352,Logan (2017),Action|Sci-Fi


In [15]:
merged_df.isnull().sum()

,0
userId,0
movieId,0
rating,0
timestamp,0
title,0
genres,0


In [16]:
merged_df.duplicated().sum()

np.int64(0)

mempertahankan genre multi-kata dalam kolom genre agar tidak dianggap sebagai dua genre yang berbeda, jadi kita akan mengganti tanda (-) menjadi (_) sebelum vektorisasi dan mengubah Kolom genres dalam format Adventure|Animation|Children menjadi Adventure Animation Children atau mengubah tanda pemisal menjadi spasi

In [17]:
merged_df['genres'] = (
    merged_df['genres']
    .str.lower()
    .str.replace('-', '_')
    .str.replace('|', ' ')
)
merged_df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),adventure animation children comedy fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),comedy romance
2,1,6,4.0,964982224,Heat (1995),action crime thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),mystery thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",crime mystery thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),drama horror thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),action crime thriller
100833,610,168250,5.0,1494273047,Get Out (2017),horror
100834,610,168252,5.0,1493846352,Logan (2017),action sci_fi


menghapus data duplikasi yang memiliki movieId yang sama. karena 1 film bisa muncul berkali-kali(karena user berbeda dalam ratings), sedangkan kita hanya ingin satu data unik untuk per film

In [18]:
movie_content_df = merged_df[['movieId', 'title', 'genres']].drop_duplicates('movieId')
movie_content_df.reset_index(drop=True, inplace=True)
movie_content_df.head()

,movieId,title,genres
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,3,Grumpier Old Men (1995),comedy romance
2,6,Heat (1995),action crime thriller
3,47,Seven (a.k.a. Se7en) (1995),mystery thriller
4,50,"Usual Suspects, The (1995)",crime mystery thriller


In [19]:
#Mengkonversi data series 'movieID' menjadi dalam bentuk list
movie_id = movie_content_df['movieId'].tolist()

#mengkonversi data series 'title' menjadi dalam bentuk list
movie_title = movie_content_df['title'].tolist()

#Mengkonversi data series 'genres' menjadi dalam bentk list
movie_genres = movie_content_df['genres'].tolist()

print(len(movie_id))
print(len(movie_title))
print(len(movie_genres))

9724
9724
9724


setelah itu buat dictionary untuk menentukan pasangan key-value pada data movie_id, movie_title, dan movie_genres yang telah kita siapkan sebelumnya.

In [20]:
movie_new = pd.DataFrame ({
    'id' : movie_id,
    'movie_title' : movie_title,
    'genres': movie_genres
})

movie_new

,id,movie_title,genres
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,3,Grumpier Old Men (1995),comedy romance
2,6,Heat (1995),action crime thriller
3,47,Seven (a.k.a. Se7en) (1995),mystery thriller
4,50,"Usual Suspects, The (1995)",crime mystery thriller
...,...,...,...
9719,160341,Bloodmoon (1997),action thriller
9720,160527,Sympathy for the Underdog (1971),action crime drama
9721,160836,Hazard (2005),action drama thriller
9722,163937,Blair Witch (2016),horror thriller


In [21]:
print(sorted(movie_new['genres'].unique()))

['(no genres listed)', 'action', 'action adventure', 'action adventure animation', 'action adventure animation children', 'action adventure animation children comedy', 'action adventure animation children comedy fantasy', 'action adventure animation children comedy imax', 'action adventure animation children comedy romance', 'action adventure animation children comedy sci_fi', 'action adventure animation children comedy sci_fi imax', 'action adventure animation children comedy western', 'action adventure animation children fantasy', 'action adventure animation children fantasy sci_fi', 'action adventure animation children sci_fi', 'action adventure animation comedy', 'action adventure animation comedy crime mystery', 'action adventure animation comedy fantasy mystery sci_fi', 'action adventure animation comedy fantasy sci_fi', 'action adventure animation comedy sci_fi', 'action adventure animation comedy thriller', 'action adventure animation crime fantasy', 'action adventure animation

# **Model Development dengan Content Based Filtering**


In [22]:
df = movie_new
df.sample(5)

,id,movie_title,genres
4974,6279,"Good Thief, The (2002)",crime drama
3916,122092,Guy X (2005),comedy war
3372,40959,"Ice Harvest, The (2005)",action comedy crime thriller
1685,3101,Fatal Attraction (1987),drama thriller
8224,4296,Love Story (1970),drama romance


# **TF-IDF Vectorizer**

In [23]:
print(sorted(df['genres'].unique()))

['(no genres listed)', 'action', 'action adventure', 'action adventure animation', 'action adventure animation children', 'action adventure animation children comedy', 'action adventure animation children comedy fantasy', 'action adventure animation children comedy imax', 'action adventure animation children comedy romance', 'action adventure animation children comedy sci_fi', 'action adventure animation children comedy sci_fi imax', 'action adventure animation children comedy western', 'action adventure animation children fantasy', 'action adventure animation children fantasy sci_fi', 'action adventure animation children sci_fi', 'action adventure animation comedy', 'action adventure animation comedy crime mystery', 'action adventure animation comedy fantasy mystery sci_fi', 'action adventure animation comedy fantasy sci_fi', 'action adventure animation comedy sci_fi', 'action adventure animation comedy thriller', 'action adventure animation crime fantasy', 'action adventure animation

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()

# Melakukan perhitungan idf pada data genres
tf.fit(df['genres'])

# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names_out()


array(['action', 'adventure', 'animation', 'children', 'comedy', 'crime',
       'documentary', 'drama', 'fantasy', 'film_noir', 'genres', 'horror',
       'imax', 'listed', 'musical', 'mystery', 'no', 'romance', 'sci_fi',
       'thriller', 'war', 'western'], dtype=object)

selanjutnya kita akan melakukan fit dan transformasi kedalam bentuk matriks

In [25]:
#Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(df['genres'])

#melihat ukuran matrix tfidf
tfidf_matrix.shape

(9724, 22)

Untuk menghasilkan vektor tf-idf dalam bentuk matriks, kita menggunakan fungsi todense()

In [26]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[0.        , 0.41681721, 0.51634045, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.54896727, 0.        , 0.        , ..., 0.54222422, 0.        ,
         0.        ],
        ...,
        [0.64123095, 0.        , 0.        , ..., 0.63335461, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.62477687, 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

kemudian mengubah matriks Tf_IDF menjadi sebuah DataFrame yang lebih mudah dibaca

In [27]:
pd.DataFrame(
    tfidf_matrix.todense(),
    columns=tf.get_feature_names_out(),
    index=df.movie_title
).sample(21, axis=1).sample(10, axis=0)

,romance,musical,mystery,comedy,thriller,crime,documentary,horror,no,imax,...,children,action,war,sci_fi,western,animation,film_noir,adventure,listed,fantasy
movie_title,,,,,,,,,,,,,,,,,,,,,
Cold Fish (Tsumetai nettaigyo) (2010),0.000000,0.000000,0.0,0.000000,0.825381,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000
TMNT (Teenage Mutant Ninja Turtles) (2007),0.000000,0.000000,0.0,0.251214,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.473974,0.343759,0.0,0.000000,0.0,0.484874,0.0,0.391415,0.0,0.45361
My Fair Lady (1964),0.481373,0.748915,0.0,0.334405,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000
Critters (1986),0.000000,0.000000,0.0,0.509792,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.860298,0.0,0.000000,0.0,0.000000,0.0,0.00000
"Peacekeeper, The (1997)",0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,1.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000
"Band's Visit, The (Bikur Ha-Tizmoret) (2007)",0.000000,0.000000,0.0,0.734280,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000
Sweet Bird of Youth (1962),0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000
Pi (1998),0.000000,0.000000,0.0,0.000000,0.574788,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.717663,0.0,0.000000,0.0,0.000000,0.0,0.00000
Amy's O (a.k.a. Amy's Orgasm) (2001),0.821275,0.000000,0.0,0.570532,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.00000


Pada tahap sebelumnya, kita telah berhasil mengidentifikasi korelasi antara movie dengan genre. Sekarang, kita akan menghitung derajat kesamaan (similarity degree) dengan teknik cosine similarity.

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

#Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix)
cosine_sim

array([[1.        , 0.15262722, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15262722, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.69543501, 0.33876915,
        0.        ],
       ...,
       [0.        , 0.        , 0.69543501, ..., 1.        , 0.39570531,
        0.        ],
       [0.        , 0.        , 0.33876915, ..., 0.39570531, 1.        ,
        0.78080334],
       [0.        , 0.        , 0.        , ..., 0.        , 0.78080334,
        1.        ]])

Pada tahapan ini, kita menghitung cosine similarity dataframe tfidf_matrix yang kita peroleh pada tahapan sebelumnya. Dengan satu baris kode untuk memanggil fungsi cosine similarity dari library sklearn, kita telah berhasil menghitung kesamaan (similarity) antar movie.


Selanjutnya, mari kita lihat matriks kesamaan setiap movie dengan menampilkan nama film dalam 5 sampel kolom (axis = 1) dan 10 sampel baris (axis=0).

In [29]:
#Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa nama resto

cosine_sim_df = pd.DataFrame(cosine_sim, index=df['movie_title'], columns=df['movie_title'] )

#Melihat similarity matrix pada setiap movie
cosine_sim_df.sample(5, axis=1). sample(10, axis=0)

movie_title,Drop Zone (1994),"Darkman II: Return of Durant, The (1995)",Impostor (2002),Dead Ringers (1988),Going the Distance (2010)
movie_title,,,,,
Inglorious Bastards (Quel maledetto treno blindato) (1978),0.309934,0.221547,0.319122,0.115660,0.000000
Butterflies Are Free (1972),0.000000,0.000000,0.230682,0.266771,0.418930
Billy Jack (1971),0.589526,0.421406,0.607003,0.219998,0.000000
Death Wish 4: The Crackdown (1987),0.589526,0.421406,0.607003,0.219998,0.000000
Blood Beach (1981),0.000000,0.409564,0.000000,0.468424,0.000000
Cool World (1992),0.000000,0.000000,0.000000,0.000000,0.201893
Hamlet (1990),0.000000,0.000000,0.339814,0.392977,0.000000
Justice League: Crisis on Two Earths (2010),0.335000,0.239465,0.597014,0.000000,0.000000
Top Five (2014),0.000000,0.000000,0.000000,0.000000,0.570532


#### **Rekomendasi**

In [30]:
def movie_recommendations(movie_title, similarity_data=cosine_sim_df, items=df[['movie_title', 'genres']], k=5):
    index = similarity_data.loc[:,movie_title].to_numpy().argpartition(
        range(-1, -k, -1))
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    closest = closest.drop(movie_title, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

In [31]:
df[df.movie_title.eq('Waterboy, The (1998)')]

,id,movie_title,genres
1598,2335,"Waterboy, The (1998)",comedy


In [32]:
#Mendapatkan rekomendasi movie yang mirip dengan 'Waterboy, The (1998)'
movie_recommendations('Waterboy, The (1998)')

,movie_title,genres
0,Private Benjamin (1980),comedy
1,Judd Apatow: The Return (2017),comedy
2,Dad's Army (2016),comedy
3,Craig Ferguson: Tickle Fight (2017),comedy
4,Dad's Army (1971),comedy


Berdasarkan hasil rekomendasi film yang memiliki genre yang sama dengan 'Waterboy, The (1998)' adalah :
1. Private Benjamin (1980)	comedy
2. Judd Apatow: The Return (2017)	comedy
3. Dad's Army (2016)	comedy
4. Craig Ferguson: Tickle Fight (2017)	comedy
5. Dad's Army (1971)	comedy

In [33]:
def evaluate_genre_precision_recall(movie_title, k=5):
    # Ambil genre dari film input
    input_row = df[df['movie_title'] == movie_title]
    if input_row.empty:
        return f"Film '{movie_title}' tidak ditemukan di dataset."

    input_genres = input_row['genres'].values[0].lower().split('|')

    # Ambil hasil rekomendasi
    recommendations = movie_recommendations(movie_title, k=k)

    # Hitung precision: berapa banyak film yang punya genre cocok
    relevant_count = 0
    for _, row in recommendations.iterrows():
        rec_genres = row['genres'].lower().split('|')
        if any(g in input_genres for g in rec_genres):
            relevant_count += 1
    precision = relevant_count / k

    # Hitung recall: apakah genre dari film input muncul di hasil rekomendasi
    recommended_genres = set()
    for _, row in recommendations.iterrows():
        rec_genres = row['genres'].lower().split('|')
        recommended_genres.update(rec_genres)

    recall = len(set(input_genres).intersection(recommended_genres)) / len(input_genres)

    return {
        'movie_input': movie_title,
        'input_genre': input_genres,
        'precision': round(precision, 2),
        'recall': round(recall, 2)
    }



In [34]:
evaluate_genre_precision_recall('Waterboy, The (1998)')

{'movie_input': 'Waterboy, The (1998)',
 'input_genre': ['comedy'],
 'precision': 1.0,
 'recall': 1.0}

dari hasil di atas yang kita dapatkan :
- Precision = 1.0 → Semua film yang direkomendasikan memang punya genre yang relevan dengan film input (comedy)

- Recall = 1.0 → Genre dari film input (comedy) tercakup di rekomendasi

selanjutnya kita akan melakukan evaluasi untuk 20 film secara acak, dan menghitung rata-rata precion dan recall

In [35]:
sample_movies = df['movie_title'].drop_duplicates().sample(20, random_state=42)

results = [evaluate_genre_precision_recall(title) for title in sample_movies]
precision_scores = [r['precision'] for r in results if isinstance(r, dict)]
recall_scores = [r['recall'] for r in results if isinstance(r, dict)]

avg_precision = sum(precision_scores) / len(precision_scores)
avg_recall = sum(recall_scores) / len(recall_scores)

print(f"Average Precision: {avg_precision:.2f}")
print(f"Average Recall: {avg_recall:.2f}")


Average Precision: 0.96
Average Recall: 1.00


Dari hasilnya di dapatkan :
- Average Precision = 0.96
Rata-rata 96% dari film yang direkomendasikan relevan (punya genre yang cocok).

- Average Recall = 1.00
Rata-rata, semua genre film input berhasil tercakup di hasil rekomendasi (genre utama selalu ketemu).

# Model Collaborative Filtering

model rekomendasi berbasis Collaborative Filtering (CF), khususnya item-based collaborative filtering dengan pendekatan Pearson Correlation sebagai ukuran kemiripan antar item (film).

In [36]:
merged_df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),adventure animation children comedy fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),comedy romance
2,1,6,4.0,964982224,Heat (1995),action crime thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),mystery thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",crime mystery thriller
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),drama horror thriller
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),action crime thriller
100833,610,168250,5.0,1494273047,Get Out (2017),horror
100834,610,168252,5.0,1493846352,Logan (2017),action sci_fi


menghitung jumlah review atau rating dari masing-masing judul film yang diberikan oleh user

In [37]:
comment_counts = merged_df["title"].value_counts().reset_index()
comment_counts.columns = ['title', 'count_user']
comment_counts.head(5)

,title,count_user
0,Forrest Gump (1994),329
1,"Shawshank Redemption, The (1994)",317
2,Pulp Fiction (1994),307
3,"Silence of the Lambs, The (1991)",279
4,"Matrix, The (1999)",278


filter film berdasarkan popularitas atau jumlah user rating

In [38]:
popular_movies = comment_counts[comment_counts["count_user"] >= 50]['title'].tolist()
popular_movies

['Forrest Gump (1994)',
 'Shawshank Redemption, The (1994)',
 'Pulp Fiction (1994)',
 'Silence of the Lambs, The (1991)',
 'Matrix, The (1999)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Jurassic Park (1993)',
 'Braveheart (1995)',
 'Terminator 2: Judgment Day (1991)',
 "Schindler's List (1993)",
 'Fight Club (1999)',
 'Toy Story (1995)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'American Beauty (1999)',
 'Usual Suspects, The (1995)',
 'Seven (a.k.a. Se7en) (1995)',
 'Independence Day (a.k.a. ID4) (1996)',
 'Apollo 13 (1995)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Godfather, The (1972)',
 'Fugitive, The (1993)',
 'Batman (1989)',
 'Saving Private Ryan (1998)',
 'Lord of the Rings: The Two Towers, The (2002)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Aladdin (1992)',
 'Fargo (1996)',


sekarang kita akan membuat matriks pengguna vs film. Dimana :
- baris = userId
- kolom = title (judul film)
- Nilai = rating yang diberikan user ke film tersebut

In [39]:
# Filter data hanya untuk film-film dalam cut_movies
filtered_df = merged_df[merged_df['title'].isin(popular_movies)]

# Buat matriks user vs film, isi nilainya adalah rating
user_movie_matrix = filtered_df.pivot_table(index='userId', columns='title', values='rating')

# Lihat hasil awal
user_movie_matrix.head()


title,10 Things I Hate About You (1999),12 Angry Men (1957),2001: A Space Odyssey (1968),28 Days Later (2002),300 (2007),"40-Year-Old Virgin, The (2005)",A.I. Artificial Intelligence (2001),"Abyss, The (1989)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),...,Willy Wonka & the Chocolate Factory (1971),"Wizard of Oz, The (1939)","Wolf of Wall Street, The (2013)",X-Men (2000),X-Men: The Last Stand (2006),X2: X-Men United (2003),You've Got Mail (1998),Young Frankenstein (1974),Zombieland (2009),Zoolander (2001)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,5.0,5.0,NaN,5.0,NaN,NaN,NaN,5.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN
4,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
user_movie_matrix.shape

(606, 450)

In [41]:
user_movie_matrix.columns

Index(['10 Things I Hate About You (1999)', '12 Angry Men (1957)',
       '2001: A Space Odyssey (1968)', '28 Days Later (2002)', '300 (2007)',
       '40-Year-Old Virgin, The (2005)', 'A.I. Artificial Intelligence (2001)',
       'Abyss, The (1989)', 'Ace Ventura: Pet Detective (1994)',
       'Ace Ventura: When Nature Calls (1995)',
       ...
       'Willy Wonka & the Chocolate Factory (1971)',
       'Wizard of Oz, The (1939)', 'Wolf of Wall Street, The (2013)',
       'X-Men (2000)', 'X-Men: The Last Stand (2006)',
       'X2: X-Men United (2003)', 'You've Got Mail (1998)',
       'Young Frankenstein (1974)', 'Zombieland (2009)', 'Zoolander (2001)'],
      dtype='object', name='title', length=450)

- membuat similarity matrix menggunakan Pearson Correlation:
Punya matriks user-film
Baris = userId
Kolom = judul film
Nilai = rating user ke film tersebut

- Hitung korelasi antar film
Kita hitung Pearson correlation antar kolom (film).
Dengan ini, kita tahu seberapa mirip pola rating antar dua film.

- Gunakan matriks korelasi ini untuk rekomendasi
Kalau user suka film A, rekomendasikan film-film yang punya korelasi tinggi dengan film A.

menghitung matrix person correlation antar film. korelaasi dihitung hanya untuk film yang memiliki minimal nilai 5, untuk user yang sama-sama memberikan rating

In [42]:
movie_similarity = user_movie_matrix.corr(method='pearson', min_periods=5)

Nilai korelasi:

- (1) artinya pola rating kedua film sangat mirip.

- (0) artinya tidak ada korelasi.

- (-1) artinya pola rating berlawanan (jika suka satu, biasanya tidak suka yang lain).



In [43]:
print(movie_similarity.head())

title                              10 Things I Hate About You (1999)  \
title                                                                  
10 Things I Hate About You (1999)                       1.000000e+00   
12 Angry Men (1957)                                    -4.382839e-17   
2001: A Space Odyssey (1968)                           -2.535161e-01   
28 Days Later (2002)                                    1.079482e-01   
300 (2007)                                              3.394157e-01   

title                              12 Angry Men (1957)  \
title                                                    
10 Things I Hate About You (1999)        -4.382839e-17   
12 Angry Men (1957)                       1.000000e+00   
2001: A Space Odyssey (1968)              4.022474e-02   
28 Days Later (2002)                     -9.787004e-03   
300 (2007)                               -2.860686e-01   

title                              2001: A Space Odyssey (1968)  \
title                

 mencari film yang paling mirip dengan film yang dipilih (movie_title). Pertama, fungsi mengambil skor korelasi antara film lain dengan film tersebut. Selanjutnya, film yang sama dengan movie_title dihapus agar tidak muncul sebagai rekomendasi. Setelah itu, skor korelasi diurutkan dari yang tertinggi ke terendah, dan akhirnya fungsi mengembalikan 5 film teratas sebagai rekomendasi

In [44]:
#def recommend_movies(movie_title, movie_similarity, top_n=5):
 #   if movie_title not in movie_similarity.columns:
  #      print("Film tidak ditemukan di data.")
   #     return

#    sim_scores = movie_similarity[movie_title].dropna()
#   sim_scores = sim_scores.drop(labels=[movie_title])
 #   top_movies = sim_scores.sort_values(ascending=False).head(top_n)

  #  print(f"Rekomendasi film mirip dengan '{movie_title}':\n")
   # for i, (title, score) in enumerate(top_movies.items(), start=1):
    #    print(f"{i}. {title} (Skor kemiripan: {score:.3f})")


In [45]:
def recommend_movies(movie_title, movie_similarity, top_n=5):
    if movie_title not in movie_similarity.columns:
        print("Film tidak ditemukan di data.")
        return pd.DataFrame()

    sim_scores = movie_similarity[movie_title].dropna()
    sim_scores = sim_scores.drop(labels=[movie_title], errors='ignore')
    top_movies = sim_scores.sort_values(ascending=False).head(top_n)

    # Return dataframe rekomendasi
    return pd.DataFrame({
        'movie_title': top_movies.index,
        'similarity_score': top_movies.values
    })


In [46]:
# Mencari rekomendasi untuk film "Forrest Gump (1994)"
recommendations = recommend_movies('Forrest Gump (1994)', movie_similarity)
print(recommendations)

                 movie_title  similarity_score
0  Mr. Holland's Opus (1995)          0.652144
1          Pocahontas (1995)          0.550118
2    Grumpier Old Men (1995)          0.534682
3          Caddyshack (1980)          0.520328
4     Few Good Men, A (1992)          0.517146


berdasarkan rekomendasi kelima film tersebut punya pola rating yang cukup mirip dengan "Forrest Gump (1994)".

Korelasi di atas 0.5 cukup tinggi untuk rekomendasi film.

Jadi, jika user suka "Forrest Gump (1994)", sistem menyarankan film-film tersebut.

In [47]:
def evaluate_cf_precision_recall(movie_title, movie_similarity, df, k=5):
    # Ambil genre film input
    input_row = df[df['movie_title'] == movie_title]
    if input_row.empty:
        return f"Film '{movie_title}' tidak ditemukan di dataset."

    input_genres = input_row['genres'].values[0].lower().split(' ')

    # Ambil rekomendasi
    recommendations = recommend_movies(movie_title, movie_similarity, top_n=k)
    if recommendations.empty:
        return f"Tidak ada rekomendasi untuk film '{movie_title}'."

    # Hitung precision
    relevant_count = 0
    for rec_title in recommendations['movie_title']:
        rec_genres = df[df['movie_title'] == rec_title]['genres'].values[0].lower().split(' ')
        if any(g in input_genres for g in rec_genres):
            relevant_count += 1
    precision = relevant_count / k

    # Hitung recall (berapa genre film input yang muncul dalam rekomendasi)
    recommended_genres = set()
    for rec_title in recommendations['movie_title']:
        rec_genres = df[df['movie_title'] == rec_title]['genres'].values[0].lower().split(' ')
        recommended_genres.update(rec_genres)
    recall = len(set(input_genres).intersection(recommended_genres)) / len(input_genres)

    return {
        'movie_input': movie_title,
        'input_genre': input_genres,
        'precision': round(precision, 2),
        'recall': round(recall, 2)
    }


In [48]:
result = evaluate_cf_precision_recall('Forrest Gump (1994)', movie_similarity, df, k=5)
print(result)


{'movie_input': 'Forrest Gump (1994)', 'input_genre': ['comedy', 'drama', 'romance', 'war'], 'precision': 1.0, 'recall': 0.75}
